In [15]:
import pandas as pd
from glob import glob
import xml.etree.ElementTree as ET
import json

In [16]:
old_path = "./data/カタログデータ - 新規画像撮影の対応表.csv"

old_df = pd.read_csv(old_path)

old_map = {}

old_map_extra = {}

for index, row in old_df.iterrows():
    value = row["画像一覧"]
    if pd.isna(value):
        continue

    id_new = row["dcterms:identifer"]

    if pd.isna(id_new):
        continue
    
    old_map[row["現在のアイテムID"]] = id_new

# old_map

with open("./tmp/old_map.json", "w") as f:
    json.dump(old_map, f)


In [22]:
files = glob("../docs/tei3/*.xml")

files.sort()

errors = []

for file in files:
    old_id = file.split("/")[-1].split(".")[0]

    if old_id.startswith("Tursko"):
        continue

    if old_id not in old_map:
        # print(old_id, old_map[old_id])
        errors.append(old_id)

        continue

    new_id = old_map[old_id]

    tree = ET.parse(file)
    root = tree.getroot()


    root.find(".//{http://www.tei-c.org/ns/1.0}title").text = new_id